In [84]:
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
from scipy import optimize as opt
from scipy.optimize import curve_fit
import pandas as pd
from astropy.cosmology import Planck18 as cosmo
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size']   = 11
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'''
\usepackage{physics}
\usepackage{siunitx}
%%% astronomical units
\DeclareSIUnit \au {au}
\DeclareSIUnit \jansky {Jk}
\DeclareSIUnit \ly {ly}
\DeclareSIUnit \parsec {pc}
\DeclareSIUnit \mag {mag}
\DeclareSIUnit \solarmass {\ensuremath{\mathit{M_{\odot}}}}
\DeclareSIUnit \yr {yr}
%%% cgs units
\DeclareSIUnit \erg {erg}
\DeclareSIUnit \cm {cm}
\DeclareSIUnit \gram {g}
'''

In [85]:
%matplotlib inline
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')

In [86]:
 results_no_nebular = Table.read('results_no_nebular_new_true.txt', format='ascii')

In [87]:
def binning(
        redshifts: npt.NDArray,
        bin_edges: npt.NDArray,
        samples_per_bin: int=10
        ) -> npt.NDArray:
    rng = np.random.default_rng()
    num_bins = len(bin_edges) - 1
    bin_indices = np.digitize(redshifts, bin_edges, right=False)
    return np.array([
        rng.choice(np.flatnonzero(bin_indices ==  i + 1), size=samples_per_bin) for i in range(num_bins)
        ])


#def log10SFR_time_Eq10_fit(log_M_star, t, a0, a1, b0, b1, b2):
 #   return (a1 * t + b1) * log_M_star + b2 * log_M_star * log_M_star + (b0 + a0 * t)
def alpha(mu, a0, a1):
   
    return a0 + a1 * mu

def beta(mu, b0, b1, b2):
   
    return b0 + b1 * mu + b2 * mu**2

def sfr_log10_model(X, a0, a1, b0, b1, b2):
  
    t, M_star = X
    mu = np.log10(M_star)

    return alpha(mu, a0, a1) * t + beta(mu, b0, b1, b2)
  


In [88]:
log_ssfr = np.log10(results_no_nebular['bayes.sfh.sfr10Myrs']/results_no_nebular['bayes.stellar.m_star'])

mask_quiescent = (log_ssfr < -10.5)
mask_starforming = (log_ssfr > -10.5)

quiescent_table = all_galaxies_table[mask_quiescent]
starforming_table = all_galaxies_table[mask_starforming]

In [89]:
starforming_table.columns

<TableColumns names=('id','bayes.sfh.sfr10Myrs','bayes.sfh.sfr10Myrs_err','bayes.stellar.m_star','bayes.stellar.m_star_err','bayes.cfht.megacam.u','bayes.cfht.megacam.u_err','bayes.paranal.vircam.H','bayes.paranal.vircam.H_err','bayes.paranal.vircam.J','bayes.paranal.vircam.J_err','bayes.paranal.vircam.Ks','bayes.paranal.vircam.Ks_err','bayes.subaru.suprime.Y','bayes.subaru.suprime.Y_err','bayes.subaru.suprime.g','bayes.subaru.suprime.g_err','bayes.subaru.suprime.i','bayes.subaru.suprime.i_err','bayes.subaru.suprime.r','bayes.subaru.suprime.r_err','bayes.subaru.suprime.z','bayes.subaru.suprime.z_err','best.chi_square','best.reduced_chi_square','best.attenuation.Av_BC','best.attenuation.Av_ISM','best.attenuation.galex.FUV','best.attenuation.generic.bessell.B','best.attenuation.generic.bessell.V','best.attenuation.mu','best.attenuation.slope_BC','best.attenuation.slope_ISM','best.sfh.age','best.sfh.age_burst','best.sfh.age_main','best.sfh.f_burst','best.sfh.tau_burst','best.sfh.tau_main'

In [90]:
# 1) Select your star‐forming sample and pull out the relevant arrays:
sf = starforming_table
z = sf['best.universe.redshift']
Mstar = sf['bayes.stellar.m_star']
sfr = sf['bayes.sfh.sfr10Myrs']

In [91]:
t = cosmo.age(z).to_value('Gyr')

In [92]:
logM = np.log10(Mstar)
logSFR = np.log10(sfr)

In [93]:
logSFR = np.log10(sfr)
# propagate SFR errors into log space
err_sfr  = sf['bayes.sfh.sfr10Myrs_err']      # your SFR‐error column
err_logS = err_sfr / (sfr * np.log(10))


In [94]:
N_bins = 10
# use percentiles to get edges that split the data into N_bins equal–N groups
bin_edges = np.percentile(z, np.linspace(0, 100, N_bins+1))

In [99]:
idxs = binning(z, bin_edges, samples_per_bin=10)

In [103]:
fit_results = []
bin_idx = np.digitize(z, bin_edges, right=True)
for i in range(1, N_bins+1):
    sel = (bin_idx == i)
    ti = t[sel]
    Mi = Mstar[sel]
    Li = logSFR[sel]

    # initial guesses for a0, a1, b0, b1, b2
    p0 = [0.20, -0.034, -26.134, 4.722, -0.1925]

    # curve_fit wants X data as a tuple:
popt, pcov = curve_fit(
    sfr_log10_model,
    (ti, Mi),
    Li,
    p0=p0,
    sigma=err_logSFR[sel],
    absolute_sigma=True,
    maxfev=5000
)



perr = np.sqrt(np.diag(pcov))
fit_results.append({
    'bin': i,
    'z_min': bin_edges[i-1],
    'z_max': bin_edges[i],
    'a0': (popt[0], perr[0]),
    'a1': (popt[1], perr[1]),
    'b0': (popt[2], perr[2]),
    'b1': (popt[3], perr[3]),
    'b2': (popt[4], perr[4]),
})

NameError: name 'err_logSFR' is not defined

In [97]:
for r in fit_results:
    print(f"Bin {r['bin']} (z = {r['z_min']:.2f}–{r['z_max']:.2f}):")
    print(f"  a0 = {r['a0'][0]:.3f} ± {r['a0'][1]:.3f}")
    print(f"  a1 = {r['a1'][0]:.3f} ± {r['a1'][1]:.3f}")
    print(f"  b0 = {r['b0'][0]:.3f} ± {r['b0'][1]:.3f}")
    print(f"  b1 = {r['b1'][0]:.3f} ± {r['b1'][1]:.3f}")
    print(f"  b2 = {r['b2'][0]:.3f} ± {r['b2'][1]:.3f}")
    print()

Bin 1 (z = 0.01–0.22):
  a0 = -0.375 ± 0.202
  a1 = 0.031 ± 0.023
  b0 = -17.014 ± 3.420
  b1 = 4.149 ± 0.505
  b2 = -0.234 ± 0.014

Bin 2 (z = 0.22–0.31):
  a0 = -0.610 ± 0.369
  a1 = 0.047 ± 0.039
  b0 = -12.703 ± 3.972
  b1 = 3.496 ± 0.468
  b2 = -0.200 ± 0.016

Bin 3 (z = 0.31–0.36):
  a0 = -3.057 ± 0.541
  a1 = 0.268 ± 0.052
  b0 = 80.330 ± 5.396
  b1 = -12.385 ± 0.559
  b2 = 0.476 ± 0.013

Bin 4 (z = 0.36–0.46):
  a0 = -1.572 ± 0.311
  a1 = 0.124 ± 0.030
  b0 = 55.632 ± 3.392
  b1 = -9.053 ± 0.412
  b2 = 0.386 ± 0.012

Bin 5 (z = 0.46–0.56):
  a0 = -8.345 ± 0.356
  a1 = 0.806 ± 0.034
  b0 = 91.251 ± 3.020
  b1 = -10.490 ± 0.359
  b2 = 0.170 ± 0.015

Bin 6 (z = 0.56–0.66):
  a0 = -6.169 ± 0.555
  a1 = 0.564 ± 0.054
  b0 = 69.913 ± 5.099
  b1 = -8.073 ± 0.627
  b2 = 0.161 ± 0.020

Bin 7 (z = 0.66–0.74):
  a0 = -6.318 ± 0.918
  a1 = 0.543 ± 0.087
  b0 = 92.568 ± 8.082
  b1 = -12.109 ± 0.930
  b2 = 0.364 ± 0.021

Bin 8 (z = 0.74–0.84):
  a0 = -3.323 ± 0.591
  a1 = 0.287 ± 0.056
  b0 

In [111]:
from astropy.cosmology import Planck18 as cosmo
from scipy.optimize import curve_fit

sf = starforming_table
z  = sf['best.universe.redshift']
M  = sf['bayes.stellar.m_star']
s  = sf['bayes.sfh.sfr10Myrs']
t  = cosmo.age(z).to_value('Gyr')

logM    = np.log10(M)
logSFR  = np.log10(s)
errSFR  = sf['bayes.sfh.sfr10Myrs_err']      # whatever your error column is
errLogS = errSFR / (s * np.log(10))          # propagate to log‐space

# equal‐N bins in z
N_bins   = 4
edges    = np.percentile(z, np.linspace(0,100,N_bins+1))
bin_idx  = np.digitize(z, edges, right=True)

# your literature starting values
p0 = [0.2, 0.0, -8.0, 0.0, 0.0]
fit_results = []
for i in range(1, N_bins+1):
    sel = (bin_idx == i)
    ti, Mi, Li = t[sel], M[sel], logSFR[sel]
    sigi       = errLogS[sel]

    popt, pcov = curve_fit(
      sfr_log10_model,
      (ti, Mi),
      Li,
      p0=p0,
    
      absolute_sigma=True,
      maxfev=5000
    )

    perr = np.sqrt(np.diag(pcov))
    fit_results.append((popt, perr))

# print them out...
for i,(p,err) in enumerate(fit_results, 1):
    print(f"Bin {i}:")
    for name,val,er in zip(("a0","a1","b0","b1","b2"), p, err):
        print(f"  {name} = {val:.3f} ± {er:.3f}")
    print()


Bin 1:
  a0 = -1.248 ± 1.338
  a1 = 0.107 ± 0.142
  b0 = 7.742 ± 24.627
  b1 = -0.111 ± 3.644
  b2 = -0.041 ± 0.114

Bin 2:
  a0 = -0.831 ± 2.074
  a1 = 0.059 ± 0.207
  b0 = 13.043 ± 30.810
  b1 = -1.463 ± 4.782
  b2 = 0.048 ± 0.194

Bin 3:
  a0 = -3.161 ± 3.523
  a1 = 0.276 ± 0.338
  b0 = 44.243 ± 43.019
  b1 = -5.589 ± 6.300
  b2 = 0.160 ± 0.235

Bin 4:
  a0 = 0.218 ± 1.898
  a1 = -0.060 ± 0.170
  b0 = -9.143 ± 45.202
  b1 = 2.496 ± 7.654
  b2 = -0.116 ± 0.326

